# Creating plots

## 0) Imports and definitions

In [38]:
# Standard
import pandas as pd
import numpy as np
from datetime import datetime

# Viz
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Other
import zeep
import json
import urllib
import requests
from unidecode import unidecode

## 1) WSDL

In [3]:
wsdl = './FachadaWSSGS.wsdl'
client = zeep.Client(wsdl=wsdl)

In [4]:
long_array_type = client.get_type('ns0:ArrayOfflong')
r = client.service.getValoresSeriesVO(long_array_type([7384]), '01/01/2000', '05/04/2021')
aux = zeep.helpers.serialize_object(r[0]['valores'])

In [5]:
df_vendas = pd.DataFrame(aux)
df_vendas['valor'] = df_vendas['valor'].apply(lambda x: int(x['_value_1']))

In [6]:
df_vendas = df_vendas.rename(columns={'valor': 'vendas'})
df_vendas = df_vendas[['mes', 'ano', 'vendas']]
df_vendas.head()

,mes,ano,vendas
0,1,2000,69984
1,2,2000,82262
2,3,2000,76748
3,4,2000,91613
4,5,2000,98154


## 3) Data POA 

In [7]:
url = 'https://dadosabertos.poa.br/api/3/action/datastore_search?resource_id=b56f8123-716a-4893-9348-23945f1ea1b9&limit=100000'

r = requests.get(url)
data_json = r.json()

In [8]:
df_acidentes = pd.DataFrame.from_dict(data_json['result']['records'])

In [9]:
df_acidentes['dia'] = df_acidentes['data'].apply(lambda x: pd.to_datetime(x, errors='coerce').day)
df_acidentes['mes'] = df_acidentes['data'].apply(lambda x: pd.to_datetime(x, errors='coerce').month)
df_acidentes['ano'] = df_acidentes['data'].apply(lambda x: pd.to_datetime(x, errors='coerce').year)
df_acidentes['tipo_acid'] = df_acidentes['tipo_acid'].apply(lambda x: unidecode(x.lower()))

In [10]:
df_acidentes = df_acidentes[['dia', 'mes', 'ano', 'tipo_acid', 'feridos', 'fatais', 'caminhao', 'moto', 'latitude', 'longitude']]
df_acidentes.head()

,dia,mes,ano,tipo_acid,feridos,fatais,caminhao,moto,latitude,longitude
0,17.0,10.0,2020.0,abalroamento,1,0,0,1,0.000000,0.000000
1,1.0,1.0,2016.0,choque,0,1,0,0,-30.134588,-51.095216
2,2.0,1.0,2016.0,queda,1,0,0,1,-30.004450,-51.200627
3,2.0,1.0,2016.0,abalroamento,1,0,0,0,-30.046960,-51.218421
4,2.0,1.0,2016.0,choque,3,0,0,0,-30.093210,-51.129850


## 4) Auxiliary functions

In [22]:
def plot_acidentes_with_filters(df, year, cat_values=[-1, -1, -1, -1]):

    cat_list = ['feridos', 'fatais', 'caminhao', 'moto']

    df_aux = df[(df['ano'] == year)]

    for col, value, in zip(cat_list, cat_values):
        if value > 0:
            df_aux = df_aux[df_aux[col] == value]
    
    df_aux = df_aux.sort_values(by='mes')
    fig = px.histogram(df_aux,
                    x="mes",
                    barmode='group',
                    width=800,
                    height=400, 
                    labels={'2016': ' '},
                    title="Acidentes por mês")
    fig.update_layout(showlegend=False, bargap=0.2)
    fig.show()

In [31]:
def plot_acidentes_per_month(df, year, month, tipo_acid=False):

    df_aux = df[(df['mes'] == month) & (df['ano'] == year)]
    if not tipo_acid:
        fig = px.histogram(df_aux, 
                           x="dia", 
                           barmode='group', 
                           width=800,
                           height=400, 
                           labels={'dia': 'Dia do mês'},
                           title="Acidentes por dia")
    else:
        fig = px.histogram(df_aux,
                           x="mes", 
                           color='tipo_acid',
                           barmode='group',
                           width=800,
                           height=400, 
                           labels={'mes': 'Mês'},
                           title="Acidentes por mês")
    fig.update_layout(showlegend=False, bargap=0.2)
    fig.show()

In [32]:
def plot_vendas_per_month(df, year):

    df_aux = df[(df['ano'] == year)]

    fig = px.bar(df_aux,
                x="mes", 
                y='vendas',
                width=800,
                height=400,
                labels={'mes': 'Mês',
                        'vendas': 'Número de vendas'},
                title="Vendas por mês")
    fig.update_layout(showlegend=False, bargap=0.2)
    fig.show()

In [81]:
def plot_acidentes_versus_vendas(df_acidentes, df_vendas, year):

    df_acid_aux = df_acidentes[df_acidentes['ano'] == year]
    df_vend_aux = df_vendas[df_vendas['ano'] == year]

    df_acid_aux = df_acid_aux[['mes', 'ano']].groupby(by='mes').count().reset_index().rename(columns={'ano': 'acidentes'})
    df_vend_aux = df_vend_aux.drop(columns=['ano']).reset_index(drop=True)

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(x=df_vend_aux['mes'], y=df_vend_aux['vendas'], name="Vendas"),
        secondary_y=True,
    )

    # Add traces
    fig.add_trace(
        go.Scatter(x=df_acid_aux['mes'], y=df_acid_aux['acidentes'], name="Acidentes"),
        secondary_y=False,
    )


    # Add figure title
    fig.update_layout(
        title_text="Número de vendas versus acidentes"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Mês")

    # Set y-axes titles
    fig.update_yaxes(title_text="Número de acidentes", secondary_y=False)
    fig.update_yaxes(title_text="Número de vendas", secondary_y=True)

    fig.show()

## 5) Plots

### 5.1) Acidentes por mês

In [34]:
plot_vendas_per_month(df_vendas, 2018)

In [35]:
plot_acidentes_with_filters(df_acidentes, 2016, [1, -1, -1, -1])

In [36]:
plot_acidentes_per_month(df_acidentes, 2017, 12, tipo_acid=True)

In [37]:
plot_acidentes_per_month(df_acidentes, 2017, 10)


In [82]:
plot_acidentes_versus_vendas(df_acidentes, df_vendas, 2017)